<h1>Computer Vision Project</h1>
<h2>Phase 1</h2>
<h3>Team 3</h3>
<ul><li>Anas Salah</li>
<li> Alaa Hamdy </li>
<li>Ahmed Amr </li>
</ul>

<h3>Import libraries and packages used</h3>

In [3]:
import os
import numpy as np
import pandas as pd
from collections import Counter
import cv2
import imutils
import json
import statistics
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

<h3>Load training images</h3>

In [4]:
images = []
labels = []

picsFolder_path = "train/train/"
with open('digitStruct.json') as f:
    data = json.load(f)

# import colored pictures
for i in range(len(data)):
    image = cv2.imread(picsFolder_path + data[i]['filename'])
    images.append(image)
    temp=[]
    for j in range(len(data[i]['boxes'])):
        temp.append(data[i]['boxes'][j]['label'])
    temp = np.array(temp)
    labels.append(temp)
# for image in os.listdir(picsFolder_path):
#     images.append(image)
print("we have",len(images),"images")



we have 33402 images


<h3>Resizing Images</h3>

In [5]:
resizedImages =[]
for image in images[:10]:
    resizedImages.append(cv2.resize(image,(100,75)))

print(len(resizedImages))


10


<h3> Creating Images Copy</h3>

In [6]:
imagesCopy = []
for resizedimage in resizedImages:
    imagesCopy.append(resizedimage)


print(len(imagesCopy))


10


<h3>Removing background from images</h3>

In [ ]:
backgroundremovedImages=[]

for resizedimage in resizedImages:

    # convert image into grayscale
    gray = cv2.cvtColor(resizedimage, cv2.COLOR_BGR2GRAY)

    # Create a background subtractor object
    bgSubtractor = cv2.createBackgroundSubtractorMOG2()

    # Apply the background subtractor to the grayscale image
    fgMask = bgSubtractor.apply(gray)

    # Apply morphological transformations to remove noise and fill holes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_OPEN, kernel)
    fgMask = cv2.morphologyEx(fgMask, cv2.MORPH_CLOSE, kernel)

    # Invert the mask to obtain the foreground
    fgMask = cv2.bitwise_not(fgMask)

    # Apply the mask to the original image
    fgImg = cv2.bitwise_and(resizedimage, resizedimage, mask=fgMask)

    # Display the result
    backgroundremovedImages.append(fgImg)

print(len(backgroundremovedImages))


10


<h3>Converting Original Images copy from BGR to Grayscale  </h3>

In [ ]:
greyImages = [] 
# Convert BGR to grayscale:
for backgroundremovedimages in backgroundremovedImages:
    greyImages.append(cv2.cvtColor(backgroundremovedimages, cv2.COLOR_BGR2GRAY)) 


print(len(greyImages))

10


<h3>Sharpening Images </h3>

In [7]:
sharpenedImages = []
for greyimage in greyImages:

    
    # apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(greyimage, (3, 3), 0)

    # apply Laplacian filter to extract edges
    laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
    laplacian = cv2.convertScaleAbs(laplacian)

    # convert original image to grayscale
    gray_resized = cv2.cvtColor(resizedimage, cv2.COLOR_BGR2GRAY)

    # apply edge sharpening using the grayscale original image and the Laplacian edges
    sharpened = cv2.convertScaleAbs(gray_resized - laplacian)
    sharpenedImages.append(sharpened)

print(len(sharpenedImages))




10


<h3>Adaptive Thresholding on Grayscale Images</h3>

In [10]:
# Set the adaptive thresholding:
windowSize = 31
windowConstant = -1

binaryImages = []


# Apply the threshold:
for greyimage in greyImages:
    binaryImages.append(cv2.adaptiveThreshold(greyimage, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,
                                        windowSize, windowConstant))
    

print(len(binaryImages))

10


<h3>Perform Connected Components on the Binary Images</h3>

In [11]:
filteredImages =[]

# Perform Connected Components:
for binaryimage in binaryImages:
    componentsNumber, labeledImage, componentStats, componentCentroids = cv2.connectedComponentsWithStats(binaryimage,
                                                                                                          connectivity=4)
    # Set the minimum pixels for the area filter to filter connected components:
    minArea = 20

    # Get the indices/labels of the remaining components based on the area stat
    # (skip the background component at index 0)
    remainingComponentLabels = [i for i in range(1, componentsNumber) if componentStats[i][4] >= minArea]
    # Filter the labeled pixels based on the remaining labels,
    # assign pixel intensity to 255 (uint8) for the remaining pixels
    # y3ni b3d de el mafood yfdal the largest connected components that hopefully contains the digits
    # one problem is that background connected components may still exist
    filteredImages.append(np.where(np.isin(labeledImage, remainingComponentLabels) == True, 255, 0).astype('uint8'))

    
print(len(filteredImages))


10


<h3>Morphological Operations - Closing on Filtered Images</h3>

In [12]:
# Set kernel (structuring element) size:
kernelSize = 3

# Set operation iterations:
opIterations = 1

# Get the structuring element:
maxKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))

closingImages = []

# Perform closing:
for filteredimage in filteredImages:
    closingImages.append(cv2.morphologyEx(filteredimage, cv2.MORPH_CLOSE, maxKernel, None, None, opIterations,
                                cv2.BORDER_REFLECT101))
    

print(len(closingImages))


10


<h3>Opening on the Closed Images </h3>

In [13]:
# Set kernel (structuring element) size:
kernelSize = 3

# Set operation iterations:
opIterations = 1

# Get the structuring element:
maxKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernelSize, kernelSize))

openingImages = []

# Perform closing:
for closingimage in closingImages:
    openingImages.append(cv2.morphologyEx(closingimage, cv2.MORPH_CLOSE, maxKernel, None, None, opIterations,
                                cv2.BORDER_REFLECT101))

<h3>Gaussian Blur on Opened Images</h3>

In [14]:
smoothedImages =[]

# perform smoothing
for closingimage in closingImages:
    smoothedImages.append(cv2.GaussianBlur(closingimage, (3, 3), 0))

<h3>Canny Edge Detection on Smoothed Images </h3>

In [15]:
edgeImages =[]

# perform smoothing
for smoothedimage in smoothedImages:
# perform canny edge detection:
    edgeImages.append(cv2.Canny(closingimage, 100, 200))


print(len(edgeImages))

10


<h3> Drawing bounding boxes on digits </h3>

In [16]:
index = 0
# Get each bounding box
# Find the big contours on the filtered image:
for edgeimage in edgeImages:
    contours, hierarchy = cv2.findContours(edgeimage, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    contours_poly = [None] * len(contours)
    # The Bounding Rectangles will be stored here:
    boundRect = []
    # Alright, just look for the outer bounding boxes:
    for i, c in enumerate(contours):
        if hierarchy[0][i][3] == -1:
            contours_poly[i] = cv2.approxPolyDP(c, 3, True)
            boundRect.append(cv2.boundingRect(contours_poly[i]))
       # Get the dimensions of the image
    # height, width, channels = imagesCopy[index].shape

    # Sort the bounding boxes based on their areas in descending order
    # boundRect = sorted(boundRect, key=lambda x: x[2] * x[3], reverse=True)

    # Create an empty list to store the selected bounding boxes
    # selectedBoundRect = []

    # Iterate through each bounding box
    # for i, box in enumerate(boundRect):
    #     # Check if the current bounding box overlaps with any of the previously selected bounding boxes
    #     overlaps = False
    #     for selectedBox in selectedBoundRect:
    #         if box[0] < selectedBox[0] + selectedBox[2] and box[0] + box[2] > selectedBox[0] and \
    #                 box[1] < selectedBox[1] + selectedBox[3] and box[1] + box[3] > selectedBox[1]:
    #             overlaps = True
    #             break
    #     # If the current bounding box doesn't overlap with any of the previously selected bounding boxes, add it to the list
    #     if not overlaps:
    #         selectedBoundRect.append(box)
        # print(len(boundRect))
    # Draw the bounding boxes on the (copied) input image:
    for i in range(len(boundRect)):
        color = (0, 255, 0)
        # area= int(boundRect[i][2]) * int(boundRect[i][3])
        # x, y, w, h = boundRect[i]

        # # Calculate the center of the bounding box
        # center_x = x + (w / 2)
        # center_y = y + (h / 2)

        # filter contours according to the area of the rectangle (parameter re5em)
        if ( int(boundRect[i][2])*int(boundRect[i][3])>100 and int(boundRect[i][2])*int(boundRect[i][3])<1000):
            cv2.rectangle(imagesCopy[index], (int(boundRect[i][0]), int(boundRect[i][1])),
                          (int(boundRect[i][0] + boundRect[i][2]), int(boundRect[i][1] + boundRect[i][3])), color, 2)
    index= index + 1

print(len(edgeImages))



10


<h3>Show Images with bounding boxes </h3>

In [17]:
for i in range(0,10):
    # plt.imshow(images[i])
    # plt.show()
    cv2.imshow('image',imagesCopy[i])
    cv2.waitKey(0)

